In [ ]:
%load_ext autoreload
%autoreload 2
import dt4dds_benchmark
import dt4dds.analysis.dataaggregation as analysis
import scipy.stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import collections

In [ ]:
exp2coderate = {
    'aeon_max': "1.81", 'aeon_high': "1.51", 'aeon_medium': "1.01",
    'fountain_max': "1.74", 'fountain_high': "1.47", 'fountain_medium': "1.00",
    'rs_max': "1.64", 'rs_high': "1.50", 'rs_medium': "1.00",
    'goldman': "0.34", 'hedges': "0.99", 'yinyang': "1.82",
}
type2name = {
    'aeon': 'DNAAeon', 'fountain': 'DNAFountain', 'rs': 'DNARS', 'goldman': 'GM', 'hedges': 'HG', 'yinyang': 'YY'
}

### read the error data from Cov1000 analysis

In [ ]:
error_data = [analysis.GroupAnalysis([
        (codec, analysis.ErrorAnalysis(f"./exp_data/{scenario}/Cov1000/full_analysis/{codec}/analysis")) for codec in [
            'aeon_max', 'aeon_high', 'aeon_medium',
            'fountain_max', 'fountain_high', 'fountain_medium',
            'rs_max', 'rs_high', 'rs_medium',
            'goldman', 'hedges', 'yinyang',
        ]
    ]) for scenario in ('bestcase', 'worstcase')
]

### retrieve overall error rates

In [ ]:
errordf = []
for i, scenario in enumerate(('bestcase', 'worstcase')):
    df = error_data[i].data['overall_error_rates'].copy()
    df['scenario'] = scenario
    errordf.append(df)

errordf = pd.concat(errordf, ignore_index=True)
errordf[['codec_type', 'codec_name']] = errordf['exp'].str.split('_', expand=True)
errordf['codec_name'] = errordf['codec_name'].fillna('default')
errordf['coderate'] = errordf['exp'].apply(lambda x: exp2coderate[x])
errordf['codec.type'] = errordf['codec_type'].apply(lambda x: type2name[x])
errordf = errordf.drop(errordf.loc[errordf['type'].isin(['insevents', 'delevents', 'subevents'])].index)

errordf

### compare total error rate

In [ ]:
toterrordf = errordf.groupby(['scenario', 'codec_type', 'codec_name']).rate.sum().reset_index()
toterrordf

In [ ]:
for scenario, yrange, dtick in (('bestcase', 0.002, 0.001), ('worstcase', 0.02, 0.01)):
    fig = dt4dds_benchmark.analysis.plotting.tiered_bar(
        errordf.loc[errordf.scenario == scenario].sort_values(['codec.type', 'coderate']),
        "codec.type",
        "coderate",
        "rate",
        color_by='type',
        color_discrete_map={'substitutions': '#e6550d', 'deletions': '#3182bd', 'insertions': '#2ca25f'},
    )
    fig.update_layout(
        barmode='stack',
        width=324,
        height=150,
        margin=dict(l=50, r=1, t=10, b=30),
        showlegend=False,
    )
    fig.update_yaxes(
        title='Error rate per nt',
        tickformat=',.1%',
        range=[0, yrange],
        dtick=dtick,
    )
    fig.add_hline(
        y=toterrordf.loc[toterrordf.scenario == scenario]['rate'].mean(),
        line_width=1,
        line_color='black',
        line_dash='dot',
    )
    print(toterrordf.loc[toterrordf.scenario == scenario]['rate'].mean())
    fig = dt4dds_benchmark.analysis.plotting.standardize_plot(fig)
    fig.update_xaxes(
        tickfont_size=28/3, 
        tickangle=0,
    )

    fig.write_image(f'./figures/error_comparison_{scenario}.svg')
    fig.write_image(f'./figures/error_comparison_{scenario}.png', scale=2)
    fig.show()

### assess coverage homogeneity

In [ ]:
covs_by_scenario = {
    'bestcase': [1000, 25, 10, 5, 2],
    'worstcase': [1000, 50, 25, 10, 5],
}

coverage_data = analysis.DistributionAnalysis({
    f"{scenario}_{cov}": f"./exp_data/{scenario}/Cov{cov}/scafstats.txt" 
    for scenario in ('bestcase', 'worstcase')
    for cov in covs_by_scenario[scenario]
})

In [ ]:
coveragedf = coverage_data.data.copy().drop(columns=['index'])
coveragedf[['scenario', 'coverage']] = coveragedf['exp'].str.split('_', expand=True)
coveragedf['codec.type'] = coveragedf['#name'].str.split('_', expand=True)[0]
coveragedf['codec.name'] = coveragedf['#name'].str.split('_', expand=True)[1]
coveragedf['codec.name'] = coveragedf['codec.name'].apply(lambda x: 'default' if x.isdigit() else x)
coveragedf['id_exp_codec'] = coveragedf['codec.type'] + '_' + coveragedf['codec.name'] + '_' + coveragedf['exp'].astype(str)
coveragedf['meanReads_by_codec'] = coveragedf['id_exp_codec'].map(coveragedf.groupby('id_exp_codec')['assignedReads'].mean())
coveragedf['x_by_codec'] = coveragedf['assignedReads'] / coveragedf['meanReads_by_codec']
coveragedf['id_codec'] = coveragedf['codec.type'] + '_' + coveragedf['codec.name']

coveragedf

In [ ]:
seqdepthdf = coveragedf.groupby(['scenario', 'codec.type', 'codec.name', 'coverage']).apply(lambda x: x['assignedReads'].sum()/x['#name'].count(), include_groups=False).reset_index()
seqdepthdf['codec_type'] = seqdepthdf['codec.type'].apply(lambda x: type2name[x])
seqdepthdf['codec_id'] = seqdepthdf['codec.type'] + '_' + seqdepthdf['codec.name']
seqdepthdf['codec_id'] = seqdepthdf['codec_id'].str.replace('_default', '')
seqdepthdf['code_rate'] = seqdepthdf['codec_id'].map(exp2coderate)
seqdepthdf['coverage'] = seqdepthdf['coverage'].astype(int)
seqdepthdf = seqdepthdf.sort_values(['codec_type', 'code_rate', 'coverage'])
seqdepthdf['coverage'] = seqdepthdf['coverage'].astype(str)

seqdepthdf

In [ ]:
colormap = {
    'bestcase': {'1000': '#d9d9d9', '25': '#bdbdbd', '10': '#969696', '5': '#636363', '2': '#252525'},
    'worstcase': {'1000': '#d9d9d9', '50': '#bdbdbd', '25': '#969696', '10': '#636363', '5': '#252525'},
}

for scenario, yrange, dtick in (('bestcase', 210, 50), ('worstcase', 210, 50)):
    fig = dt4dds_benchmark.analysis.plotting.tiered_bar(
        seqdepthdf.loc[seqdepthdf.scenario == scenario].sort_values(['codec_type', 'code_rate']),
        "codec_type",
        "code_rate",
        0,
        color_by='coverage',
        color_discrete_map=colormap[scenario],
    )
    fig.update_layout(
        width=324,
        height=150,
        margin=dict(l=50, r=1, t=10, b=30),
        showlegend=False,
    )
    fig.update_yaxes(
        title='Sequencing depth',
        range=[0, yrange],
        dtick=dtick
    )
    fig.add_hline(
        y=30,
        line_width=1,
        line_color='red',
    )
    fig.add_hline(
        y=seqdepthdf[0].mean(),
        line_width=1,
        line_color='black',
        line_dash='dot',
    )

    fig = dt4dds_benchmark.analysis.plotting.standardize_plot(fig)
    fig.update_xaxes(
        tickfont_size=28/3, 
        tickangle=0,
    )

    fig.write_image(f'./figures/seqdepth_comparison_{scenario}.svg')
    fig.write_image(f'./figures/seqdepth_comparison_{scenario}.png', scale=2)
    fig.show()

### assess sequence dropout after downsampling

In [ ]:
downsampleddf = []

for id in coveragedf.id_exp_codec.unique():
    expdf = coveragedf.loc[coveragedf.id_exp_codec == id].copy().reset_index(drop=True)
    seqs = [expdf['#name'][k] for k in range(len(expdf)) for _ in range(int(expdf['assignedReads'].iloc[k]))]
    seq_set = set(expdf['#name'].values)
    for _ in range(10):
        sampled_seqs = collections.Counter(np.random.choice(seqs, size=int(30*len(expdf)), replace=False))
        missing_seqs = seq_set - set(sampled_seqs.keys())
        downsampleddf.append({
            'id_exp_codec': id,
            'dropout_n': len(missing_seqs),
            'dropout_p': len(missing_seqs)/len(seq_set),
        })

downsampleddf = pd.DataFrame(downsampleddf)

downsampleddf

In [ ]:
dropoutdf = downsampleddf.groupby('id_exp_codec')['dropout_p'].agg(['mean', 'std']).reset_index()
dropoutdf[['codec.type', 'codec.name', 'scenario', 'coverage']] = dropoutdf['id_exp_codec'].str.split('_', expand=True)
dropoutdf['codec_type'] = dropoutdf['codec.type'].apply(lambda x: type2name[x])
dropoutdf['codec_id'] = dropoutdf['codec.type'] + '_' + dropoutdf['codec.name']
dropoutdf['codec_id'] = dropoutdf['codec_id'].str.replace('_default', '')
dropoutdf['code_rate'] = dropoutdf['codec_id'].map(exp2coderate)
dropoutdf['coverage'] = dropoutdf['coverage'].astype(int)
dropoutdf = dropoutdf.sort_values(['scenario', 'codec_type', 'code_rate', 'coverage'])
dropoutdf['coverage'] = dropoutdf['coverage'].astype(str)

dropoutdf.head()

In [ ]:
colormap = {
    'bestcase': {'1000': '#d9d9d9', '25': '#bdbdbd', '10': '#969696', '5': '#636363', '2': '#252525'},
    'worstcase': {'1000': '#d9d9d9', '50': '#bdbdbd', '25': '#969696', '10': '#636363', '5': '#252525'},
}

for scenario, yrange, dtick in (('bestcase', 0.3, 0.1), ('worstcase', 0.3, 0.1)):
    fig = dt4dds_benchmark.analysis.plotting.tiered_bar(
        dropoutdf.loc[dropoutdf.scenario == scenario].sort_values(['codec_type', 'code_rate']),
        "codec_type",
        "code_rate",
        "mean",
        color_by='coverage',
        color_discrete_map=colormap[scenario],
    )
    fig.update_layout(
        # barmode='stack',
        width=324,
        height=150,
        margin=dict(l=50, r=1, t=10, b=30),
        showlegend=False,
    )
    fig.update_yaxes(
        tickformat=',.0%',
        title='Sequence dropout',
        range=[0, yrange],
        dtick=dtick
    )
    fig = dt4dds_benchmark.analysis.plotting.standardize_plot(fig)
    fig.update_xaxes(
        tickfont_size=28/3, 
        tickangle=0,
    )

    fig.write_image(f'./figures/dropout_comparison_{scenario}.svg')
    fig.write_image(f'./figures/dropout_comparison_{scenario}.png', scale=2)
    fig.show()

### show evolution of coverage bias

In [ ]:
plotdf = coveragedf.copy()
plotdf['coverage'] = -plotdf['coverage'].astype(int)
plotdf = plotdf.sort_values(['scenario', 'codec.type', 'coverage'])

for scenario, yrange, dtick in (('bestcase', 300, 0.1), ('worstcase', 300, 0.1)):
    fig = px.histogram(
        plotdf.loc[plotdf.scenario == scenario],
        x='x_by_codec',
        facet_row='id_codec',
        facet_col='coverage',
        category_orders={
            'id_codec': ['aeon_medium', 'aeon_high', 'aeon_max', 'fountain_medium', 'fountain_high', 'fountain_max', 'rs_medium', 'rs_high', 'rs_max', 'goldman_default', 'hedges_default', 'yinyang_default'],
        },
    )

    fig.update_xaxes(range=[0, 3])
    fig.update_xaxes(title="Normalized coverage", row=1)

    fig.update_yaxes(range=[0, yrange])
    fig.update_yaxes(title="Count", col=1)

    fig.update_traces(xbins_size=0.1, xbins_start=0, xbins_end=3, marker_line_width=0, marker_line_color='black')

    fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[1]))

    fig.update_layout(
        width=680,
        height=800,
        margin=dict(l=0, r=10, t=20, b=0),
        showlegend=False,
    )
    fig = dt4dds_benchmark.analysis.plotting.standardize_plot(fig)

    fig.write_image(f'./figures/coverage_evolution_{scenario}.svg')
    fig.write_image(f'./figures/coverage_evolution_{scenario}.png', scale=2)
    fig.show()

### assess standard deviation across 1000x pools

In [ ]:
stddf = coveragedf.loc[coveragedf.coverage == '1000'].groupby(['scenario', 'codec.type', 'codec.name'])['x'].std().reset_index()
stddf